In [1]:
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import pandas as pd
from PIL import Image

2024-08-18 12:10:18.116226: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
# Resized images
train_dir = '../data/train' 
valid_dir = '../data/valid'
test_dir = '../data/test'

In [15]:
# Generador de datos con normalizacion y aumentación solo para el entrenamiento, esto ayuda a generalizar mejor.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_test_datagen = ImageDataGenerator(rescale=1./255)

# Crear los generadores
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = valid_test_datagen.flow_from_directory(
    valid_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = valid_test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 10682 images belonging to 2 classes.
Found 3562 images belonging to 2 classes.
Found 3561 images belonging to 2 classes.


In [16]:
# Seleccionar y cargar el modelo preentrenado
from tensorflow.keras.applications import DenseNet121, EfficientNetB0

# Cambiar este valor para probar diferentes modelos
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [17]:
# Congelar las capas del modelo base
base_model.trainable = False

# Añadir capas superiores personalizadas
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(2, activation='softmax')(x)  # Suponiendo 2 clases: Melanoma y NotMelanoma

# Crear el modelo completo
model = models.Model(inputs=base_model.input, outputs=outputs)

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Mostrar la arquitectura del modelo
model.summary()

# Guardar el mejor modelo durante el entrenamiento
checkpoint = ModelCheckpoint('../models/best_melanomaornot_model_02.keras',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

# Parar temprano si no hay mejora
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

# Reducir la tasa de aprendizaje si no hay mejora
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, min_lr=1e-7)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d      │ (None, 230, 230,  │          0 │ input_layer_1[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,408 │ zero_padding2d[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_1    │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1               │ (None, 56, 56,    │          0 │ zero_padding2d_1… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │        256 │ pool1[0][0]       │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_relu │ (None, 56, 56,    │          0 │ conv2_block1_0_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      8,192 │ conv2_block1_0_r… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        512 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,864 │ conv2_block1_1_r… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_concat │ (None, 56, 56,    │          0 │ pool1[0][0],      │
│ (Concatenate)       │ 96)               │            │ conv2_block1_2_c… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_bn   │ (None, 56, 56,    │        384 │ conv2_block1_con… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_relu │ (None, 56, 56,    │          0 │ conv2_block2_0_b… │
│ (Activation)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_1_conv │ (None, 56, 56,    │     12,288 │ conv2_block2_0_r

 Total params: 7,563,330 (28.85 MB)

 Trainable params: 525,826 (2.01 MB)

 Non-trainable params: 7,037,504 (26.85 MB)

In [18]:
# Entrenar el modelo
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=30,
    callbacks=[checkpoint, early_stopping, reduce_lr]
)



Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


333/333 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7407 - loss: 1.0932
Epoch 1: val_loss improved from inf to 0.58200, saving model to models/best_melanomaornot_model_02.keras
333/333 ━━━━━━━━━━━━━━━━━━━━ 1334s 4s/step - accuracy: 0.7410 - loss: 1.0926 - val_accuracy: 0.9350 - val_loss: 0.5820 - learning_rate: 1.0000e-04
Epoch 2/30
  1/333 ━━━━━━━━━━━━━━━━━━━━ 16:50 3s/step - accuracy: 0.7812 - loss: 0.8217

2024-08-17 09:29:02.969374: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)
2024-08-17 09:29:03.970474: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]



Epoch 2: val_loss improved from 0.58200 to 0.45796, saving model to models/best_melanomaornot_model_02.keras
333/333 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.7812 - loss: 0.8217 - val_accuracy: 1.0000 - val_loss: 0.4580 - learning_rate: 1.0000e-04
Epoch 3/30
333/333 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.8932 - loss: 0.6220
Epoch 3: val_loss did not improve from 0.45796
333/333 ━━━━━━━━━━━━━━━━━━━━ 1393s 4s/step - accuracy: 0.8932 - loss: 0.6218 - val_accuracy: 0.9378 - val_loss: 0.4675 - learning_rate: 1.0000e-04
Epoch 4/30
  1/333 ━━━━━━━━━━━━━━━━━━━━ 16:10 3s/step - accuracy: 0.9688 - loss: 0.4837

...

333/333 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.9220 - loss: 0.4257
Epoch 9: val_loss did not improve from 0.28888

Epoch 9: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
333/333 ━━━━━━━━━━━━━━━━━━━━ 2005s 6s/step - accuracy: 0.9220 - loss: 0.4257 - val_accuracy: 0.9406 - val_loss: 0.3817 - learning_rate: 1.0000e-04
Epoch 10/30
  1/333 ━━━━━━━━━━━━━━━━━━━━ 16:59 3s/step - accuracy: 0.8125 - loss: 0.6702

2024-08-17 11:13:37.442260: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]



Epoch 10: val_loss did not improve from 0.28888
333/333 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8125 - loss: 0.6702 - val_accuracy: 0.9000 - val_loss: 0.3803 - learning_rate: 1.0000e-05
Epoch 11/30


2024-08-17 11:13:38.385794: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


333/333 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.9227 - loss: 0.4041
Epoch 11: val_loss did not improve from 0.28888
333/333 ━━━━━━━━━━━━━━━━━━━━ 2424s 7s/step - accuracy: 0.9227 - loss: 0.4041 - val_accuracy: 0.9389 - val_loss: 0.3876 - learning_rate: 1.0000e-05
Epoch 11: early stopping
Restoring model weights from the end of the best epoch: 6.


In [24]:
# Evaluar el modelo en el conjunto de pruebas
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

# Calcular el número de steps exactos para cubrir todas las muestras
steps = int(np.ceil(test_generator.samples / test_generator.batch_size))

# Calcular las predicciones con el número de steps correcto
predictions = model.predict(test_generator, steps=steps, verbose=1)

# Asegúrate de que no falten imágenes al final del proceso
predicted_classes = np.argmax(predictions, axis=1)

# Imprimir el tamaño de la salida de predicciones
print(f"Predictions shape: {predictions.shape}")
print(f"Predicted classes length: {len(predicted_classes)}")
print(f"True classes length: {len(true_classes)}")

# Comparar las longitudes de true_classes y predicted_classes
true_classes = test_generator.classes
print(f'Longitud de true_classes: {len(true_classes)}')
print(f'Longitud de predicted_classes: {len(predicted_classes)}')

# Solo generar el reporte si las longitudes coinciden
if len(true_classes) == len(predicted_classes):
    report = classification_report(true_classes, predicted_classes, target_names=class_labels)
    print(report)
else:
    print("Las longitudes de true_classes y predicted_classes no coinciden. No se puede generar el reporte.")

# Mostrar la matriz de confusión
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("Confusion Matrix")
print(conf_matrix)

111/111 ━━━━━━━━━━━━━━━━━━━━ 393s 4s/step - accuracy: 0.9469 - loss: 0.4141
Test Loss: 0.46514827013015747
Test Accuracy: 0.9313063025474548
112/112 ━━━━━━━━━━━━━━━━━━━━ 377s 3s/step
Predictions shape: (3561, 2)
Predicted classes length: 3561
True classes length: 3561
Longitud de true_classes: 3561
Longitud de predicted_classes: 3561
              precision    recall  f1-score   support

    Melanoma       0.99      0.87      0.93      1781
 NotMelanoma       0.89      0.99      0.94      1780

    accuracy                           0.93      3561
   macro avg       0.94      0.93      0.93      3561
weighted avg       0.94      0.93      0.93      3561

Confusion Matrix
[[1555  226]
 [  18 1762]]


In [25]:
# Guardar el modelo actualizado
model.save('../models/best_melanomaornot_model_02_final.keras')